In [12]:
pip install pymongo pandas scipy matplotlib seaborn statsmodels scikit-posthocs 



[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from pymongo import MongoClient
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import kruskal
import scikit_posthocs as sp

In [14]:
# MongoDB connection details
mongo_uri = "mongodb://localhost:27017/"
database_name = "research"
collection_name = "performance-refactorings"

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[database_name]
collection = db[collection_name]

# Load performance refactorings data
performance_refactorings = list(collection.find({}, {"_id": 0, "repo_fullname": 1, "type":1, }))
print(f"Total number of performance refactorings: {len(performance_refactorings)}")


Total number of performance refactorings: 19646


In [15]:
df = pd.DataFrame(performance_refactorings)

# Group by 'repo_fullname' and 'type', and count occurrences
grouped_df = df.groupby(["repo_fullname", "type"]).size().reset_index(name="counts")

# Save the grouped data to a CSV file
grouped_df.to_csv("../../data/rq1/grouped_refactorings.csv", index=False)

In [18]:
# Descriptive Statistics
frequencies = grouped_df.groupby("type")["counts"].sum()
average_frequencies = grouped_df.groupby("type")["counts"].mean()

# Combine frequencies and average frequencies into one DataFrame
combined_df = pd.DataFrame(
    {"frequency": frequencies, "average_frequency": average_frequencies}
)

# Reset the index to include 'type' as a column
combined_df.reset_index(inplace=True)

# Sort the DataFrame by total_counts in descending order
combined_df = combined_df.sort_values(by="average_frequency", ascending=False)

# Save to a CSV file
combined_df.to_csv("../../data/rq1/descriptive_analysis.csv", index=False)

print("Frequencies:\n", frequencies)
print("\n")
print("Average Frequencies:\n", average_frequencies)

print(
    "Combined Frequencies and Average Frequencies saved to 'combined_frequencies.csv'"
)